In [ ]:
# 📘 FX Reinforcement Learning System（簡易版）
"""
- 作成者：fxinsightslab
- バージョン：v1.0
- 公開日：2025年4月xx日
- ライセンス：個人利用限定（再配布・商用利用・改変配布禁止）

---

## 📄 利用に関する注意事項

- このノートブックは学習・研究・検証目的で提供されています。
- 実際の売買判断やトレード運用は自己責任の下で行ってください。
- 本テンプレートを改変しての再配布、または第三者への提供・販売は禁止です。

> 💬 ご質問や不具合の報告は、X（旧Twitter）[@fxinsightslab](https://x.com/fxinsightslab) までお気軽にどうぞ！

---

✅ 以下のセルから順次実行すると最後の評価まで行えるようになっています。
"""

In [1]:
# データ準備
################################################################
# 指定されたプロジェクトルートからノートブックを実行し、
# MT5からGOLDのM5データを取得・前処理して表示する
################################################################
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from sklearn.preprocessing import StandardScaler
import os


# --- プロジェクトルートを自動検出する関数 ---
def find_project_root(target_folder_name="FX Reinforcement Learning System（簡易版）"):
    current_path = os.getcwd() # 現在の作業ディレクトリを取得
    while True:
        # 現在のフォルダ名がターゲット名と一致するか確認
        if os.path.basename(current_path) == target_folder_name:
            return current_path # 一致すればそのパスを返す
        parent = os.path.dirname(current_path)# 一つ上の階層へ移動
        if parent == current_path:
            # ルートまで探索しても見つからなければエラーを投げる
            raise FileNotFoundError(f"{target_folder_name} フォルダが上位ディレクトリに見つかりませんでした。")
        current_path = parent# 次のループでは上位フォルダを探索

try:# プロジェクトルートのパスを取得し、指定ノートブックを実行
    base_path = find_project_root()  
    target_path = os.path.join(base_path, "src/utils/data_preparation.ipynb").replace("\\", "/")
    %run "$target_path"# ノートブックを実行して関数などを読み込み
except FileNotFoundError as e:
    print(e)# 見つからなければエラーメッセージを表示


# --- MT5からデータ取得 ---
mt5.initialize()  # MT5との接続を初期化
symbol = "GOLD"  # 取得する銘柄を指定
# GOLDの5分足データを「現在の最新の足」から遡って1000本取得
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 1000, 100000)  # 0 は「現在時刻に最も近いバー（=最新バー）を起点」という意味
mt5.shutdown()  # MT5との接続を終了

# --- データフレームに変換・整形 ---
df = pd.DataFrame(rates)  # MT5データをDataFrame形式に変換
df["time"] = pd.to_datetime(df["time"], unit="s")  # UNIX時間を日時形式に変換

# --- 前処理実行（data_preparation.ipynbで定義された関数） ---
df = prepare_data(df)  # 指定関数でテクニカル指標などを追加・整形

# --- 結果表示（先頭5行） ---
print(df.head())  # データの先頭を表示して確認

                  time      open      high       low     close  tick_volume  \
49 2023-12-04 18:15:00 -1.410565 -1.410908 -1.413262 -1.412014         1063   
50 2023-12-04 18:20:00 -1.412092 -1.407153 -1.413115 -1.408814          907   
51 2023-12-04 18:25:00 -1.408745 -1.409911 -1.417816 -1.416212         1240   
52 2023-12-04 18:30:00 -1.416495 -1.414956 -1.418903 -1.414773         1076   
53 2023-12-04 18:35:00 -1.414470 -1.412609 -1.420049 -1.419059         1055   

    spread  real_volume    volume  raw_close    SMA_20    EMA_50    RSI_14  \
49      26            0  1.114432    2025.81 -1.382311 -1.337241 -1.873442   
50      27            0  0.733753    2026.90 -1.384852 -1.340045 -1.653285   
51      26            0  1.546356    2024.38 -1.388468 -1.343029 -1.880135   
52      31            0  1.146155    2024.87 -1.391894 -1.345839 -1.776553   
53      26            0  1.094910    2023.41 -1.395795 -1.348708 -1.912200   

        MACD    ATR_14  BB_upper  BB_lower       OBV  zi

In [2]:
# ========== ボラティリティ分析AI用の強化学習環境 ==========
################################################################
# 状態空間	TrendEnvと同様の13指標（ボラティリティ系も含む）
# 行動空間	0=保留、1=リスク回避、2=エントリー促進
# 報酬設計	ATRによるペナルティ（リスク回避）／価格変動による報酬（エントリー促進）
# 拡張性	価格変化のしきい値、標準偏差の導入、報酬感度などを調整可能
################################################################
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class VolatilityEnv(gym.Env):
    def __init__(self, data):
        super(VolatilityEnv, self).__init__()
        self.data = data.reset_index(drop=True)  # インデックスをリセット
        self.current_step = 0  # ステップカウンタ初期化
        self.max_steps = len(data) - 1  # 最大ステップ数をデータ長で定義

        # --- 状態空間：13特徴量（TrendEnvと統一）---
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32
        )

        # --- 行動空間 ---
        # 0: 保留、1: リスク回避、2: エントリー促進
        self.action_space = spaces.Discrete(3)

    # --- 現在の状態を返す（13次元ベクトル）---
    def _get_state(self):
        row = self.data.iloc[self.current_step]
        return np.array([
            row["close"],
            row["SMA_20"],
            row["RSI_14"],
            row["MACD"],
            row["ATR_14"],
            row["BB_upper"],
            row["BB_lower"],
            row.get("volume", 0.0),  # volume がない場合は 0 にする
            row["open"],
            row["high"],
            row["low"],
            row["EMA_50"],
            row["OBV"]
        ], dtype=np.float32)

    # --- 環境のリセット ---
    def reset(self, seed=None):
        self.current_step = 0  # 最初のステップに戻る
        return self._get_state(), {}

    # --- 1ステップ進める処理 ---
    def step(self, action):
        reward = 0  # 初期報酬
        terminated = False
        truncated = False

        # --- リスク回避行動（ボラティリティが高いときに有効）---
        if action == 1:
            # ATR（平均的な価格変動幅）に基づいてペナルティ（=市場不安定）
            reward = -self.data["ATR_14"].iloc[self.current_step] * 0.2

        # --- エントリー促進行動（価格が大きく動いたときに有効）---
        elif action == 2:
            price_diff = (
                self.data["close"].iloc[self.current_step]
                - self.data["close"].iloc[self.current_step - 1]
            )
            reward = price_diff * 0.5  # 感度を調整（0.5倍）

        # --- ステップ更新 ---
        self.current_step += 1
        if self.current_step >= self.max_steps:
            terminated = True  # データ末尾に達したら終了

        # --- 次の状態と報酬を返す ---
        next_state = self._get_state()
        return next_state, reward, terminated, truncated, {}


In [4]:
import os
import random
import datetime
import glob
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback

print("torch.cuda.is_available():", torch.cuda.is_available())
print("torch.cuda.device_count():", torch.cuda.device_count())

# ✅ データの一部をランダム抽出
def get_random_slice(df, length=100):
    if len(df) <= length:
        return df.copy()
    start_idx = random.randint(0, len(df) - length)
    return df.iloc[start_idx:start_idx + length].reset_index(drop=True)

# ✅ 最新モデルを自動で読み込む
def load_latest_model(model_prefix="volatility_ai_ppo", models_dir="models"):
    files = glob.glob(os.path.join(models_dir, f"{model_prefix}_*.zip"))
    if not files:
        raise FileNotFoundError(f"'{model_prefix}_*.zip' に一致するモデルが見つかりません")
    latest_file = max(files, key=os.path.getctime)
    print(f"📥 最新モデル読み込み中: {latest_file}")
    model = PPO.load(latest_file, device="cpu")
    return model

# ✅ エントロピーによる early stop + 日付付き保存 + 再学習
class EntropyStopCallback(BaseCallback):
    def __init__(self, df, model_name, entropy_threshold=0.20, max_retries=30, verbose=1):
        super().__init__(verbose)
        self.df = df
        self.model_name = model_name
        self.entropy_threshold = entropy_threshold
        self.retry_count = 0
        self.max_retries = max_retries

    def _save_model_with_timestamp(self, model):
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"models/{self.model_name}_{timestamp}.zip"
        model.save(save_path)
        print(f"💾 モデル保存: {save_path}")

    def _on_step(self) -> bool:
        entropy = self.logger.name_to_value.get("train/entropy_loss")
        if entropy is not None and abs(entropy) < self.entropy_threshold:
            print(f"\n🛑 Entropy ({entropy:.4f}) < {self.entropy_threshold:.2f} → 学習停止")
            return False
        return True

    def _on_training_end(self) -> None:
        self._save_model_with_timestamp(self.model)
        if self.retry_count < self.max_retries:
            self.retry_count += 1
            print(f"🔁 新しいデータで再学習を開始（{self.retry_count}回目）")
            new_df = get_random_slice(self.df)
            new_env = VolatilityEnv(new_df)
            monitored_env = Monitor(new_env)
    
            # ✅ モデルを再利用して再学習（環境だけ入れ替え）
            self.model.set_env(monitored_env)
            self.model.learn(total_timesteps=100000, callback=self)
    

# ✅ モデル読み込み or 新規作成
def load_or_create_model(env, model_name="volatility_ai_ppo", force_new=False, verbose=1):
    if not force_new:
        try:
            model = load_latest_model(model_prefix=model_name)
            model.set_env(env)
            print(f"✅ モデル再利用完了")
            return model
        except FileNotFoundError:
            print("⚠ モデルが見つからないため、新規作成します。")

    print(f"🆕 モデル '{model_name}' を新規作成します。")
    return PPO(
        "MlpPolicy",
        env,
        verbose=verbose,
        policy_kwargs=dict(net_arch=[dict(pi=[128, 128], vf=[128, 128])]),
        device="cpu"
    )

# ✅ 環境初期化と学習開始
sliced_df = get_random_slice(df)
env = VolatilityEnv(sliced_df)
check_env(env)
monitored_env = Monitor(env)

model_name = "volatility_ai_ppo"
force_train = False
model = load_or_create_model(monitored_env, model_name=model_name, force_new=force_train)
callback = EntropyStopCallback(df, model_name, entropy_threshold=0.10)

try:
    print("🟢 学習開始（Ctrl+Cで手動中断できます）")
    model.learn(total_timesteps=100000, callback=callback)
except KeyboardInterrupt:
    print("\n🛑 手動停止 → モデル保存中...")
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model.save(f"models/{model_name}_{timestamp}.zip")
    print(f"✅ 中断時のモデル保存完了: models/{model_name}_{timestamp}.zip")
else:
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model.save(f"models/{model_name}_{timestamp}.zip")
    print(f"✅ 学習完了・モデル保存済み: models/{model_name}_{timestamp}.zip")


torch.cuda.is_available(): True
torch.cuda.device_count(): 1
📥 最新モデル読み込み中: models\volatility_ai_ppo_20250509_195328.zip
Wrapping the env in a DummyVecEnv.
✅ モデル再利用完了
🟢 学習開始（Ctrl+Cで手動中断できます）
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | 0.734    |
| time/              |          |
|    fps             | 468      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------

🛑 Entropy (-0.0947) < 0.10 → 学習停止
💾 モデル保存: models/volatility_ai_ppo_20250509_195416.zip
🔁 新しいデータで再学習を開始（1回目）
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | 2.42     |
| time/              |          |
|    fps             | 497      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
-------------------

In [12]:
import os
import glob
import zipfile
from stable_baselines3 import PPO

# --- プロジェクトルートを自動検出（既存の関数） ---
try:
    base_path = find_project_root()
    models_dir = os.path.join(base_path, "src/agents/models")
    model_prefix = "volatility_ai_ppo"

    # --- モデルファイル群の中から最新ファイルを検索 ---
    pattern = os.path.join(models_dir, f"{model_prefix}_*.zip")
    model_files = glob.glob(pattern)

    if not model_files:
        print(f"❌ モデルファイルが見つかりません: {pattern}")
    else:
        # --- 最新ファイルを選択 ---
        latest_model_path = max(model_files, key=os.path.getctime)
        print("✅ 最新モデルファイル:", latest_model_path)

        try:
            # --- モデル読み込み ---
            model = PPO.load(latest_model_path)
            print("✅ モデルの読み込みに成功しました！")

            # --- zipファイルの内容確認 ---
            with zipfile.ZipFile(latest_model_path, 'r') as zip_ref:
                print("📦 含まれるファイル一覧:")
                for name in zip_ref.namelist():
                    print("  -", name)

        except Exception as e:
            print("❌ モデルの読み込みに失敗しました:", e)

except FileNotFoundError as e:
    print("❌ プロジェクトルートの検出に失敗しました:", e)


✅ 最新モデルファイル: C:\Users\kotak\FX Reinforcement Learning System（簡易版）\src/agents/models\volatility_ai_ppo_20250508_221836.zip
✅ モデルの読み込みに成功しました！
📦 含まれるファイル一覧:
  - data
  - pytorch_variables.pth
  - policy.pth
  - policy.optimizer.pth
  - _stable_baselines3_version
  - system_info.txt


In [13]:
# ========== PnLベースのモデル評価関数（ロング＆ショート両対応） ==========

def evaluate_model_with_pnl(model, env, steps=500):
    obs, _ = env.reset()  # 環境初期化
    df = env.data.reset_index(drop=True)  # データフレームをリセット

    position = None  # 現在のポジション（None / "long" / "short"）
    entry_price = 0  # エントリー価格
    entry_step = 0  # エントリーステップ番号
    capital = 10000  # 評価開始時の仮想資金
    equity_curve = [capital]  # 資産の推移を記録
    trade_log = []  # 各トレードの詳細記録用

    for i in range(steps):
        current_price = df["raw_close"].iloc[env.current_step]  # 実価格を取得

        # --- モデルにより行動を予測 ---
        action, _ = model.predict(obs)
        action = int(action)
        obs, reward, terminated, truncated, _ = env.step(action)

        # === エントリー処理 ===
        if action == 2 and position is None:
            # シンプルにステップの偶奇でポジション決定（ロング／ショート）
            position = "long" if i % 2 == 0 else "short"
            entry_price = current_price
            entry_step = i

        # === クローズ処理 ===
        elif action == 1 and position is not None:
            if position == "long":
                pnl = current_price - entry_price
            elif position == "short":
                pnl = entry_price - current_price
            else:
                pnl = 0

            # キャピタルに損益を反映
            capital += pnl

            # トレード記録を追加
            trade_log.append({
                "position": position,
                "entry_step": entry_step,
                "exit_step": i,
                "entry_price": entry_price,
                "exit_price": current_price,
                "pnl": pnl
            })
            position = None  # ポジションクローズ

        equity_curve.append(capital)  # 資産推移を記録

        # 終了条件
        if terminated or truncated:
            break

    # ========== 結果の集計・出力 ==========
    total_trades = len(trade_log)
    win_trades = sum(1 for t in trade_log if t["pnl"] > 0)
    total_pnl = sum(t["pnl"] for t in trade_log)
    max_drawdown = max(
        [max(equity_curve[:i+1]) - equity_curve[i] for i in range(len(equity_curve))]
    )

    # --- 評価レポート表示 ---
    print("📊 トレード損益ベース評価（ロング & ショート）")
    print(f"・評価期間: {df['time'].iloc[0]} ～ {df['time'].iloc[env.current_step]}")
    print(f"・総トレード数: {total_trades}")
    print(f"・勝率: {win_trades / total_trades * 100:.2f}%" if total_trades > 0 else "・勝率: 0%")
    print(f"・累積損益: {total_pnl:.2f}")
    print(f"・最終キャピタル: {capital:.2f}")
    print(f"・最大ドローダウン: {max_drawdown:.2f}")


# ========== 評価実行（Volatility AIモデルに対して） ==========
model = model  # 既にロード済みの学習モデル
eval_env = VolatilityEnv(df)  # VolatilityEnv 環境で df（raw_close 列が必要）を使用
evaluate_model_with_pnl(model, eval_env, steps=500)


📊 トレード損益ベース評価（ロング & ショート）
・評価期間: 2023-12-01 21:00:00 ～ 2023-12-05 16:40:00
・総トレード数: 41
・勝率: 48.78%
・累積損益: -44.63
・最終キャピタル: 9955.37
・最大ドローダウン: 62.75
